In [4]:
% matplotlib inline
from __future__ import print_function

from collections import Counter
import json
import os
import codecs
import glob
import math
import csv

import MeCab
import lda
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import scipy.sparse as sparse
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# 共通関数

In [5]:
#形態素解析
def xxratax_morph(text):
    #Mecabで形態素解析
    mecab = MeCab.Tagger("-Ochasen")
    node = mecab.parseToNode(text)
    out_words = []
    
    #名詞だけ抽出（out_wordsに文字列を格納）
    while node:
        word_type = node.feature.split(",")[0]
        if word_type in ["名詞"]:
            out_words.append(node.surface)
        node = node.next

    return out_words    

# データの読み込みと形態素解析

In [6]:
fpath = 'data/'
fext = '.txt'
file_list = glob.glob(fpath+'*'+fext)

In [7]:
doc_data = {}
vocabs = []

for fn in file_list:
    f = open(fn,'r')
    text = f.read()
    filename = fn.replace(fpath,'').replace(fext,'')
    doc_data[filename] = xxratax_morph(text)
    for w in doc_data[filename]:
        vocabs.append(w)
    print(filename)
    f.close()

all_doc_index = doc_data.keys()
all_doc_index_ar = np.array(list(all_doc_index))

predict_third
test4
test2
test3
test


In [8]:
#重複を消すためにsetしてlistにする
all_vocab = list(set(vocabs))
vocab_num = len(all_vocab)

#インデックスで使用するためにnumpy配列化
all_vocab_ar = np.array(all_vocab)

print('重複削除後の単語数: ', vocab_num)

重複削除後の単語数:  474


In [9]:
#スパース行列に格納
sps_ar = sparse.lil_matrix((len(all_doc_index), vocab_num), dtype=np.int)

total_elements_num = 0
for i in range(len(all_doc_index)):
    doc_idx = all_doc_index_ar[i]
    row_data = Counter(doc_data[doc_idx])
    print(doc_idx)
    print(row_data)
    
    for word in row_data.keys():
        word_idx = np.where(all_vocab_ar == word)[0][0]
        sps_ar[i, word_idx] = row_data[word]
        total_elements_num += 1

print('Total elements num :', total_elements_num)

predict_third
Counter({'契約': 20, '条': 16, '交換': 15, '物件': 15, '乙': 12, '所有': 12, '権': 10, '移転': 10, '登記': 10, '地': 9, '甲': 8, '大田原': 7, '市': 7, '締結': 7, '１': 7, '負担': 7, 'とき': 7, '２': 6, '財産': 6, '金': 6, '損害': 6, '甲乙': 5, '両者': 5, '渡': 5, '番': 5, '受財': 5, '産': 5, '価格': 5, '者': 5, '４': 4, '書': 4, '以下': 4, '相互': 4, '義務': 4, '所': 4, '等': 4, 'もの': 4, 'こと': 4, '３': 3, '次': 3, '履行': 3, '目': 3, '嘱託': 3, '必要': 3, '承諾': 3, '規定': 3, '当該': 3, '賠償': 3, '解除': 3, '日': 3, '管轄': 3, '平成': 3, '年': 3, '月': 3, '字': 3, '号': 2, '信義': 2, '誠実': 2, 'とおり': 2, '積': 2, '備': 2, '考': 2, '㎡': 2, '円': 2, '差金': 2, '請求': 2, '保証': 2, 'それぞれ': 2, '相手方': 2, '前項': 2, 'すべて': 2, '引渡し': 2, '帰': 2, 'かし': 2, 'その他': 2, 'ため': 2, '費用': 2, '疑義': 2, '決定': 2, 'うえ': 2, '栃木': 2, '県': 2, '通': 2, '市長': 2, '津久井': 2, '様式': 1, '関係': 1, '土地': 1, '目的': 1, '条項': 1, '放棄': 1, '５': 1, '免除': 1, '６': 1, '際': 1, '提出': 1, '登録': 1, '免許': 1, '税': 1, '経費': 1, '７': 1, '前条': 1, '項': 1, '完了': 1, '時': 1, '善良': 1, '管理': 1, '注意': 1, '無償': 1, '保管': 1, '危険': 1, 

# LDAでトピックモデル化

In [10]:
#LDA（トピック数は5）
topic_num = 5
model1 = LatentDirichletAllocation(n_topics = topic_num,
                                   doc_topic_prior=0.001,
                                   topic_word_prior=0.5,
                                   max_iter=5,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=0)

#スパース行列をLDAでトピック化
model1.fit(sps_ar)

#正規化
normalize_components = model1.components_ / model1.components_.sum(axis=0)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [11]:
#LDA内のトピックと単語の組み合わせ
n_top_words = 20
for topic_idx, topic in enumerate(normalize_components):
    print('Topic #%d:' % topic_idx)
    print(' '.join([all_vocab_ar[i] for i in
                    topic.argsort()[:-n_top_words - 1:-1]]))
    print()

Topic #0:
交換 所有 移転 登記 地 物件 大田原 負担 市 締結 権 財産 産 番 受財 渡 価格 両者 相互 甲乙

Topic #1:
起算 訳書 記 結果 修正 措置 保持 遂行 経過 限り それ 割合 会 13 出納 示 音波 スケジュール 原因 違約

Topic #2:
者 受託 調査 2 暴力団 1 報酬 等 建築 委託 住宅 業務 事務所 士 場合 員 インスペクション 書面 事項 本件

Topic #3:
公課 配分 不足 経過 不完全 対比 不正 調停 新旧 情報 会 ． 取得 設計 特約 裁判 成立 承継 以上 本条

Topic #4:
経費 計画 実施 事業 甲 明細 乙 納入 代表 ２ 変更 期間 支払 金額 書 第三者 株式会社 使用 請求 日



In [12]:
#文書あたりのトピックの重要度
doc_topic_data = model1.transform(sps_ar)

#正規化
normalize_doc_topic_data = doc_topic_data / doc_topic_data.sum(axis=1, keepdims=True)

In [13]:
#LDA内の文章とトピックの組み合わせ
df=pd.DataFrame()
df['filename']= doc_data.keys()

i=0
t='topic'

while i < topic_num:
    df[t+str(i)]=''
    i=i+1

i=0
for dfile in doc_data.keys():
    #print(dfile)
    for topic_idx, prob in enumerate(normalize_doc_topic_data[i]):
        df[t+str(topic_idx)][df['filename']==dfile] = prob
        #print('Topic #%d: probality: %f' % (topic_idx, prob))
    i=i+1

df

,filename,topic0,topic1,topic2,topic3,topic4
0,predict_third,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06
1,test4,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06
2,test2,1.02774e-06,1.02774e-06,1.02774e-06,1.02774e-06,0.999996
3,test3,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06
4,test,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06


# 教師データの読み込み

In [14]:
inputfile = pd.read_csv('教師データ.csv')

In [15]:
dataset = pd.merge(df, inputfile, how="left", on="filename")
dataset

,filename,topic0,topic1,topic2,topic3,topic4,category
0,predict_third,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,NaN
1,test4,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,4.0
2,test2,1.02774e-06,1.02774e-06,1.02774e-06,1.02774e-06,0.999996,1.0
3,test3,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,2.0
4,test,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,2.0


# 学習データとテストデータの準備

In [16]:
#学習データ・テストデータの識別フラグ
def set_datatype(category):
    if math.isnan(category):
        return 1
    else:
        return 0

dataset['datatype'] = dataset['category'].apply(set_datatype)
dataset['category'] = dataset['category'].replace(np.NaN,0)
dataset

,filename,topic0,topic1,topic2,topic3,topic4,category,datatype
0,predict_third,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,0.0,1
1,test4,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,4.0,0
2,test2,1.02774e-06,1.02774e-06,1.02774e-06,1.02774e-06,0.999996,1.0,0
3,test3,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,2.0,0
4,test,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,2.0,0


In [17]:
#目的変数と説明変数の設定
target_col = 'category'
exclude_cols = ['id', 'filename','datatype','category']
feature_cols = [col for col in dataset.columns if col not in exclude_cols]

In [18]:
train_data = dataset[dataset['datatype']==0]
test_data = dataset[dataset['datatype']==1]

y_train = np.array(train_data[target_col])
X_train = np.array(train_data[feature_cols])
X_test = np.array(test_data[feature_cols])

# 機械学習

In [19]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [20]:
#ランダムフォレストで学習
rf = RandomForestClassifier(random_state=1234)
rf.fit(X_train, y_train)

#学習データの予測誤差
y_train_pred = rf.predict(X_train)
rf_mse = mean_squared_error(y_train, y_train_pred)
print('RandomForest MSE: ', rf_mse)

train_data['predict'] = y_train_pred
train_data

RandomForest MSE:  1.0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,filename,topic0,topic1,topic2,topic3,topic4,category,datatype,predict
1,test4,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,4.0,0,4.0
2,test2,1.02774e-06,1.02774e-06,1.02774e-06,1.02774e-06,0.999996,1.0,0,1.0
3,test3,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,2.0,0,2.0
4,test,1.36798e-06,1.36798e-06,0.999995,1.36798e-06,1.36798e-06,2.0,0,4.0


In [21]:
#ランダムフォレストで予測
y_test_pred = rf.predict(X_test)

test_data['predict'] = y_test_pred
test_data

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,filename,topic0,topic1,topic2,topic3,topic4,category,datatype,predict
0,predict_third,0.999992,2.02427e-06,2.02427e-06,2.02427e-06,2.02427e-06,0.0,1,2.0


# レコメンデーション（Cos類似度）